### If the data is dirty it has to be cleaned

In [2]:
# Python modules
import flickrapi
import pandas as pd
import os
import json
from dotenv import load_dotenv
load_dotenv()

# Import additional functions
from flickr_functions import df_remove_dupes

# Keys needed for API access
api_key = os.getenv('flickr_api_key')
api_secret = os.getenv('flickr_api_secret')

# Flickr API object
flickr = flickrapi.FlickrAPI(api_key, api_secret, format='parsed-json')

# Data directory used to store CVS files
data_dir = './data/'

In [53]:
# Load EXIF dataframe from CVS file
df_exif = pd.read_csv(data_dir + 'df_photo_exif_final.csv', index_col=[0])

In [54]:
df_ids = pd.read_csv(data_dir + 'df_photo_ids_final.csv', index_col=[0])

In [48]:
df_ids

,Unnamed: 0,id,owner,secret,title,ispublic
92381,96039,53135764057,154104874@N04,73444af4c2,It's Like A New Boat BTS,1
92382,96040,53136358506,154104874@N04,7df431308b,It's Like A New Boat BTS,1
92383,96041,53136832023,154104874@N04,0ec992074c,It's Like A New Boat BTS,1
92384,96042,53136762650,154104874@N04,6c4fa436bd,It's Like A New Boat BTS,1
92385,96043,53136559689,154104874@N04,28bdbc9855,It's Like A New Boat BTS,1
...,...,...,...,...,...,...
89914,92321,53134644569,198001330@N02,3eca57011b,202324_DYNMXC-IMG_9446,1
89915,92322,53134844710,198001330@N02,b3162a8a7d,202324_DYNMXC-IMG_9449,1
89916,92323,53134644494,198001330@N02,e6743a340d,202324_DYNMXC-IMG_9450,1
89917,92324,53134439631,198001330@N02,1e7b92398d,202324_DYNMXC-IMG_9459,1


In [55]:
print(len(df_ids))
df_ids = df_ids[~df_ids.id.isin(df_exif.id)]
print(len(df_ids))

208712
125701


In [41]:
# This is too slow
print(f'length before: {len(df_photo_ids)}')
for i_ex, r_ex in df_photo_exif.iterrows():
    for i_id, r_id in df_photo_ids.iterrows():
        if r_id.get('id') == r_ex.get('id'):
            print(f'found id {r_id.get("id")} in both')
            df_photo_ids = df_photo_ids.drop(i_id)
print(f'length after: {len(df_photo_ids)}')

length before: 253639
found id 52733753731 in both
found id 52734237723 in both
found id 52734166625 in both
found id 52733753246 in both
found id 52733753146 in both
found id 52734166280 in both
found id 52734237113 in both
found id 52734166015 in both
found id 52734007289 in both
found id 52734165715 in both
found id 52733227857 in both
found id 52734165330 in both
found id 52733227667 in both
found id 52733751831 in both
found id 52734006244 in both
found id 52734006124 in both
found id 52734235723 in both
found id 52734005924 in both
found id 52734235508 in both
found id 52734164475 in both
found id 52733750936 in both
found id 52734005459 in both
found id 52733750741 in both
found id 52734234908 in both
found id 52734005259 in both
found id 52733750486 in both
found id 52734234743 in both
found id 52733750246 in both
found id 52734234558 in both
found id 52734234383 in both
found id 52734234158 in both
found id 52733225597 in both
found id 52733225402 in both
found id 52734003899 

KeyboardInterrupt: 

In [15]:
df_photo_ids.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 253639 entries, 89919 to 91101
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  253639 non-null  int64 
 1   id          253639 non-null  int64 
 2   owner       253639 non-null  object
 3   secret      253639 non-null  object
 4   title       251289 non-null  object
 5   ispublic    253639 non-null  int64 
dtypes: int64(3), object(3)
memory usage: 13.5+ MB


In [8]:
df_photo_exif = df_remove_dupes(df_photo_exif)

Found 157170 duplicates. Dataframe is now 118002 rows long.


In [63]:
df_tmp = df_photo_exif.groupby(df_photo_exif.country).count()['id'].sort_values(ascending=False)
df_tmp

country
na                181474
United States       9087
United Kingdom      8579
tbd                 8113
Belgium             3847
                   ...  
Åland Islands          3
Cameroon               2
Uzbekistan             2
Bolivia                2
Kuwait                 2
Name: id, Length: 130, dtype: int64

In [4]:
df_photo_exif[(df_photo_exif['city'] == 'King of Prussia')]['id'].count()

8220

In [5]:
counter = 0
for index, row in df_photo_exif.iterrows():
    if row.get('city') == 'King of Prussia':
        df_photo_exif.loc[index, 'country'] = 'tbd'
        df_photo_exif.loc[index, 'admin_lvl1'] = 'tbd'
        df_photo_exif.loc[index, 'admin_lvl2'] = 'tbd'
        df_photo_exif.loc[index, 'city'] = 'tbd'
print(counter)


0


In [4]:
df_photo_exif.columns

Index(['id', 'Image Width', 'Image Height', 'Compression', 'Make', 'Model',
       'Orientation', 'Software', 'Date and Time (Modified)', 'Exposure',
       'Aperture', 'ISO Speed', 'Date and Time (Original)',
       'Date and Time (Digitized)', 'Flash', 'Focal Length', 'White Balance',
       'owner', 'secret', 'title', 'lat', 'lon', 'acc'],
      dtype='object')

In [ ]:
df_photo_exif.sample(5)

In [ ]:
df_photo_exif = df_photo_exif.drop(['Creator Tool'], axis=1)

In [ ]:
df_photo_exif["id"] = pd.to_numeric(df_photo_exif["id"])

In [ ]:
df_photo_exif.sample()

In [ ]:
df_photo_exif.info()

In [ ]:
# Drop all rows that have 10 or less entries per row
df_photo_exif = df_photo_exif.drop(df_photo_exif[df_photo_exif.count(axis='columns') <= 10].index)

In [15]:
bla = len(df_photo_exif[df_photo_exif.count(axis='columns') <= 10])
print(bla)

0


In [ ]:
# Only keep rows that have the Make field filled
df_photo_exif = df_photo_exif[df_photo_exif['Make'].notnull()]

In [ ]:
df_photo_exif.sample(10)

In [ ]:
df_ids.to_csv(data_dir + 'df_photo_ids_final.csv')

In [9]:
df_photo_exif.to_csv('./data/df_photo_exif_final.csv')